# Simulating IPFS and IPNS Systems

This notebook provides a Python-based simulation of IPFS (InterPlanetary File System) and IPNS (InterPlanetary Naming System) to test various linking strategies for storing and retrieving IPAROs.

The notebook uses three classes to simulate these systems:
- **IPARO**: Represents the storage object on IPFS.
- **IPNS**: Keeps track of the latest capture for different websites.
- **IPFS**: Simulates the hashing, storage, and retrieval of IPARO objects.

The goal of the simulation is to test various linking strategies.

In [3]:
# Importing the necessary libraries
import hashlib
import random

## IPARO Object

**Properties:**
- `CID`: The CID (Content Identifier) generated by IPFS.
- `Data`: The data of the capture.
- `Linked Node CID(s)`: The CID(s) of the nodes linked to it.

**Functions:**
- `get_cid`: Returns the CID of the IPARO.
- `get_linked_cids`: Returns the CID(s) of the linked node(s).
- `get_content`: Returns the content of the IPARO.
- `__str__`: Returns a string representation of the IPARO object.

In [4]:
class IPARO:
    def __init__(self, cid: str, linked_cids: list, content: str, timestamp: str):
        """
        Initialize an IPARO object with its CID, linked CID(s), and content.

        Args:
            cid (str): The CID of the IPARO.
            linked_cids (list): List of CIDs of linked nodes.
            content (str): The content of the IPARO.
        """
        self.__cid = cid
        self.__linked_cids = linked_cids
        self.__content = content
        self.__timestamp = timestamp

    def get_cid(self) -> str:
        '''
        Returns the CID of the IPARO.

        Returns:
            str: The CID of the IPARO.
        '''
        return self.__cid

    def get_linked_cids(self) -> list:
        '''
        Returns the CID(s) of linked nodes.

        Returns:
            list: List of linked node CIDs.
        '''
        return self.__linked_cids

    def get_content(self) -> str:
        '''
        Returns the content of the IPARO.

        Returns:
            str: The content stored in the IPARO.
        '''
        return self.__content

    def get_timestamp(self) -> str:
        '''
        Returns the timestamp of the IPARO

        Returns:
            str: The timestamp in seconds since the epoch
        '''
        return self.__timestamp

    def __str__(self):
        '''
        Returns a string representation of the IPARO object.

        Returns:
            str: A string containing the CID, linked CID(s), and content of the IPARO.
        '''
        iparo = {
            "CID": self.__cid,
            "Content": self.__content,
            "Linked CID(s)": self.__linked_cids,
            "Timestamp": self.__timestamp,
        }
        return str(iparo)

## IPNS Object

**Description:**
- The IPNS class stores and maps the latest CID of a website.
- Tracks the number of operations (get and update) performed.

**Functions:**
- `update`: Updates the latest CID of a website.
- `get_cid`: Retrieves the CID of the latest capture for a website.
- `get_counts`: Returns the number of operations performed.
- `reset_counts`: Resets the counters for operations.

In [5]:
class IPNS:
    def __init__(self):
        """
        Initialize the IPNS object with an empty hashmap for storing CIDs 
        and counters for tracking operations.
        """
        self.data = {}
        self.update_count = 0
        self.get_count = 0

    def update(self, url, cid):
        '''
        Updates the latest CID for a given URL.

        Args:
            url (str): The URL of the website.
            cid (str): The CID of the latest capture.
        '''
        self.update_count += 1
        self.data[url] = cid

    def get_cid(self, url) -> str:
        '''
        Retrieves the latest CID for a given URL.

        Args:
            url (str): The URL of the website.

        Returns:
            str: The CID of the latest capture for the given URL.
        '''
        self.get_count += 1
        return self.data[url]

    def get_counts(self) -> dict:
        '''
        Returns the number of update and get operations performed.

        Returns:
            dict: Dictionary with the counts of update and get operations.
        '''
        counts = {"get": self.get_count, "update": self.update_count}
        return counts

    def reset_counts(self):
        """
        Resets the operation counters.
        """
        self.update_count = 0
        self.get_count = 0

## IPFS Object

**Description:**
- The IPFS class stores the nodes and simulates the hashing, storage, and retrieval operations.
- Tracks the number of operations (hash, store, retrieve).

**Functions:**
- `hash`: Hashes the content of a node to generate its CID.
- `store`: Stores a node with its CID.
- `retrieve`: Retrieves a node using its CID.
- `get_counts`: Returns the number of operations performed.
- `reset_counts`: Resets the counters for operations.

In [6]:
class IPFS:
    def __init__(self):
        '''
        Initialize the IPFS object with an empty hashmap for storing nodes
        and counters for tracking operations.
        '''
        self.data = {}
        self.hash_count = 0
        self.store_count = 0
        self.retrieve_count = 0

    def hash(self, content: str) -> str:
        '''
        Hashes the content to generate a CID.

        Args:
            content (str): The content of the node.

        Returns:
            str: The generated CID.
        '''
        sha256_hash = hashlib.sha256(content.encode()).hexdigest()
        self.hash_count += 1
        return 'Qm' + sha256_hash[:34]

    def store(self, cid: str, node: IPARO):
        '''
        Stores a node with its CID.

        Args:
            cid (str): The CID of the node.
            node (IPARO): The IPARO object to store.
        '''
        self.store_count += 1
        self.data[cid] = node

    def retrieve(self, cid) -> IPARO:
        '''
        Retrieves a node using its CID.

        Args:
            cid (str): The CID of the node to retrieve.

        Returns:
            IPARO: The retrieved IPARO object.
        '''
        self.retrieve_count += 1
        return self.data[cid]

    def get_counts(self) -> dict:
        '''
        Returns the number of hash, store, and retrieve operations performed.

        Returns:
            dict: Dictionary with counts of hash, store, and retrieve operations.
        '''
        counts = {"hash": self.hash_count, "store": self.store_count,
                  "retrieve": self.retrieve_count}
        return counts

    def reset_counts(self):
        """
        Resets the operation counters.
        """
        self.hash_count = 0
        self.store_count = 0
        self.retrieve_count = 0

    def reset_data(self):
        self.data = {}

    def get_data(self) -> dict:
        """Returns the data stored by IPFS (for debugging)."""
        return self.data

## Initialization and Operation Tracking

Here, we initialize the IPFS and IPNS objects and define a helper function `get_op_counts()` to display the number of operations performed.


In [7]:
# Initializing the simulated IPFS and IPNS
ipfs = IPFS()
ipns = IPNS()


def get_op_counts():
    '''
    Displays the number of operations performed by IPNS and IPFS.
    '''
    print("Number of operations IPNS performed:")
    print(ipns.get_counts())
    print("Number of operations IPFS performed:")
    print(ipfs.get_counts())

## Testing Different Linking Strategies

### 1. Linking to Only the Previous Node

In this test, each node will link only to the previous node in the chain. This strategy will be used to simulate a simple sequential storage system.


#### Storing Nodes

In [8]:
import time

# Testing parameters
NODE_NUM = 100
URL = "example.com"

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)


# Automate the creation of additional nodes
for i in range(1, NODE_NUM):
    content = f"Node {i}"
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    linked_cids = [ipns.get_cid(URL)]  # Link to the previous node
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

get_op_counts()  # Output operation counts

Number of operations IPNS performed:
{'get': 99, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 0}


#### Retrieving Nodes

The following section tests retrieval of nodes by simulating a random node search.


In [9]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Traverse back through the linked nodes to find the target
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
while node.get_content() != target_content:
    node = ipfs.retrieve(node.get_linked_cids()[0])

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 86
Found node: {'CID': 'Qm22c64d2739efefae7a8632e8c752d4f7fa', 'Content': 'Node 86', 'Linked CID(s)': ['Qm8e1780134b0ad64739805235549ef295f4'], 'Timestamp': 1737219680.2209897}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 14}


### 2. Linking to all previous nodes

In this test, each node will link to all the previous nodes in the chain.

#### Storing Nodes

In [10]:
# Resetting IPFS from the last test
ipfs.reset_data()

# Testing parameters
NODE_NUM = 100
URL = "example.com"

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

From here, there are 2 ways of creating a new node for this linking strategy, since this is a simulation, no data corruption can happen but that might not be true in practice. When retrieving the latest node which should contain the CIDs of all the previous node, two scenarios can happen:
1. The data is intact and the CIDs in the list is "correct" (which we really can't know for sure) and we can just add it to the new node we're creating
2. The data is corrupt and one or more of the CIDs is wrong or unfinished, in which case we have to recheck every CID to rebuild a new list of linked CIDs (not to mention fixing all the corrupted nodes)

So, for the purpose of this simulation, we will perform a check for every CID in the linked CID list of an IPARO to simulate the worst case scenario every time

In [11]:
# To automate adding the rest of the nodes
for i in range(2, NODE_NUM):
    content = "Node "+str(i)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    linked_cids = []
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids = latest_node_linked_cids
    for link_cid in latest_node_linked_cids:
        ipfs.retrieve(link_cid)
        # Checking and repairing nodes goes here
    linked_cids.append(latest_node_cid)
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

# print(ipfs.get_data())
get_op_counts()  # Output operation counts

Number of operations IPNS performed:
{'get': 100, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 4963}


In this worst case scenario, where we have to retrieve and verify every CIDs in the linked CIDs of an IPARO, the retrieve count goes to almost 5000 (if we're storing 100 nodes)\
Of course this trade off makes it really easy to navigate to all the nodes just from the latest nodes

#### Retrieving nodes

The following section tests retrieval of nodes by simulating a random node search.

In [12]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Traverse back through the linked nodes to find the target
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
for linked_cid in linked_cids:
    node = ipfs.retrieve(linked_cid)
    if node.get_content() == target_content:
        break

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 11
Found node: {'CID': 'Qm1b1e163a87a876ac72c1363d3642f163e4', 'Content': 'Node 11', 'Linked CID(s)': ['Qm74075d1df32399978be1482d2da515ef93', 'Qme1dac373c7aa880e9a5c913ba3331f7764', 'Qm8a07fb0d9976860d66b6b6869be9a54a3b', 'Qm2b164f0d584b104b4e111e9b7f3a53988f', 'Qm24f765e759304b8ac1988883d20873a04b', 'Qmb06a3905344b4f67555363e04146c1f9d0', 'Qmba753741ba99805d94dbb78a0846b315b3', 'Qmacdbe90430821e7f42b46f623e5e245325', 'Qmb564301b57c04e5ab57d90236bb79f2858', 'Qmd51502456bd0000bcd7b7c2827c352624a', 'Qm0912585122fe715c8d7881b54d9b6d0209', 'Qm1b1e163a87a876ac72c1363d3642f163e4', 'Qmfefcdc54347b5f084c0d2d3a15d0c22433', 'Qme83ddae631b6478d09aee57f95fb6d143d', 'Qm397a79520a7249c41d437e3baa24a5aafa', 'Qmbe05ee30b6d03a897fe384f2d15731c65f', 'Qm239310cab7418d506c1c7c571625d9a731', 'Qmc8d80ceaae0ecb26bce5ad769cdc74a990', 'Qmce3be00c1f7eb6f2e86f2cdc7808f4eccc', 'Qmcebef27b348dc52aaeddbb860186679715', 'Qma4a576d9a17d81be2c6f2b8eb02f5c70fc', 'Qm357f88fc7fedc87956

### 3. Linking to previous and first node

In this test, each node will link to the previous node and the first node in the chain.

#### Storing Nodes

In [28]:
# Resetting IPFS from the last test
ipfs.reset_data()

# Testing parameters
NODE_NUM = 100
URL = "example.com"

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

In [29]:
# To automate creating and adding the remaining nodes
for i in range(2, NODE_NUM):
    content = "Node "+str(i)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    linked_cids = []
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids.append(latest_node_linked_cids[0])
    linked_cids.append(latest_node_cid)
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

# print(ipfs.get_data())
get_op_counts()  # Output operation counts

Number of operations IPNS performed:
{'get': 100, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 139}


#### Retrieving Nodes

In [30]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Check if the first node is the desired node then search the other nodes
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
first_node_cid = linked_cids[0]
first_node = ipfs.retrieve(first_node_cid)
if first_node.get_content() == target_content:
    print(f"Found node: {node}")
else:
    while True:
        node = ipfs.retrieve(linked_cids[1])
        linked_cids = node.get_linked_cids()
        if node.get_content() == target_content:
            break

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 90
Found node: {'CID': 'Qmc0187f924869a07a418bd8e2b99b5a9d3f', 'Content': 'Node 90', 'Linked CID(s)': ['Qmbe77950e4b572465af416fdb2ba7694dd5', 'Qmf3786d0f2006c25a313de74e4aa14992d4'], 'Timestamp': 1737222945.0940273}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 11}


### 4. Linking to K-previous and first node

In this test, each node will link to K previous node and the first node in the chain.

#### Storing Nodes

In [16]:
# Resetting IPFS from the last test
ipfs.reset_data()

# Testing parameters
NODE_NUM = 100
URL = "example.com"
K = 5

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

In [17]:
# To automate creating and adding the remaining nodes
for i in range(2, NODE_NUM):
    content = "Node "+str(i)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids = []
    linked_cids.append(latest_node_linked_cids[0])
    if len(latest_node_linked_cids) == K+1:
        linked_cids.extend(latest_node_linked_cids[2:])
    else:
        linked_cids.extend(latest_node_linked_cids[1:])
    linked_cids.append(latest_node_cid)
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

get_op_counts()  # Output operation counts

Number of operations IPNS performed:
{'get': 100, 'update': 100}
Number of operations IPFS performed:
{'hash': 100, 'store': 100, 'retrieve': 174}


#### Retrieving Nodes

In [18]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Check if the first node is the desired node then search the other nodes
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
first_node_cid = linked_cids[0]
first_node = ipfs.retrieve(first_node_cid)
if first_node.get_content() == target_content:
    print(f"Found node: {node}")
else:
    # run through the rest of the linked cids list to check
    for cid in linked_cids:
        node = ipfs.retrieve(cid)
        if node.get_content() == target_content:
            break
    # Get the linked cids of the node at index 1
    node = ipfs.retrieve(linked_cids[1])
    linked_cids = node.get_linked_cids()
    while True:
        # Flag to determine if we should exit the while loop
        found = False

        # Run through the rest of the linked CIDs list to check
        for cid in linked_cids[1:]:
            node = ipfs.retrieve(cid)
            if node.get_content() == target_content:
                found = True
                break

        if found:
            break  # Exit the while loop if the node was found

        if len(linked_cids) >= 2:
            node = ipfs.retrieve(linked_cids[1])
            linked_cids = node.get_linked_cids()
        else:
            print('Can\'t find node')
            break

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 58
Found node: {'CID': 'Qm9549b5b013a743ac04c8c23d392a173a9e', 'Content': 'Node 58', 'Linked CID(s)': ['Qm2088883a7713e18efe5e994f9b94a3e877', 'Qm6b740d154133c853c583aede5fbddbfe9d', 'Qme96d8950e1c4d6fbf01c96f3dbb440d5e3', 'Qm2f7be594a77e71d7cb7a14cc4c560e6585', 'Qm5575928951d74aaa6783280052d238dda7', 'Qmd84ca238f043cae0222708fd58a285d438'], 'Timestamp': 1737219680.3335264}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 56}


### 5. Linking to K-random and first node

In this test, each node will link to a random K previous node and the first node in the chain.

#### Storing Nodes

In [31]:
import random

# Resetting IPFS from the last test
ipfs.reset_data()

# Testing parameters
NODE_NUM = 30
URL = "example.com"
Kmin = 5
Kmax = 10

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

# To automate creating and adding the remaining nodes
for i in range(2, NODE_NUM):
    content = "Node "+str(i)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids = []
    linked_cids.append(latest_node_linked_cids[0])
    K = random.randint(Kmin, Kmax)
    # Check if the number of linked CIDs is greater than K and add K-1 random linked CIDs
    if len(latest_node_linked_cids) > K:
        linked_cids.extend(random.sample(latest_node_linked_cids[1:], K-1))
    # If the number of linked CIDs is less than K add all the linked CIDs
    else:
        linked_cids.extend(latest_node_linked_cids[1:])
    linked_cids.append(latest_node_cid)
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    print("K = ", K+1)
    print("Length of linked_cids: ", len(linked_cids))
    print(node)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

get_op_counts()  # Output operation counts

K =  11
Length of linked_cids:  2
{'CID': 'Qmc5a67246ae7de78d5f879d48b7d3fbb8d6', 'Content': 'Node 2', 'Linked CID(s)': ['Qmc06f910f97a2c73fb6fbde6e3b1e7ce756', 'Qm8f80bd8bb8fc158e89c4b9c8ad9559a0fa'], 'Timestamp': 1737222966.4240603}
K =  8
Length of linked_cids:  3
{'CID': 'Qm3bc86c60ab0c7e4c9a72c1a3ddcd955e8c', 'Content': 'Node 3', 'Linked CID(s)': ['Qmc06f910f97a2c73fb6fbde6e3b1e7ce756', 'Qm8f80bd8bb8fc158e89c4b9c8ad9559a0fa', 'Qmc5a67246ae7de78d5f879d48b7d3fbb8d6'], 'Timestamp': 1737222966.4245827}
K =  8
Length of linked_cids:  4
{'CID': 'Qm24fbe9dc21ffbf1176b019dbb512279fde', 'Content': 'Node 4', 'Linked CID(s)': ['Qmc06f910f97a2c73fb6fbde6e3b1e7ce756', 'Qm8f80bd8bb8fc158e89c4b9c8ad9559a0fa', 'Qmc5a67246ae7de78d5f879d48b7d3fbb8d6', 'Qm3bc86c60ab0c7e4c9a72c1a3ddcd955e8c'], 'Timestamp': 1737222966.4245827}
K =  11
Length of linked_cids:  5
{'CID': 'Qmd16583be1e8339044a7e40029c196b1004', 'Content': 'Node 5', 'Linked CID(s)': ['Qmc06f910f97a2c73fb6fbde6e3b1e7ce756', 'Qm8f80bd8bb8fc1

#### Retrieving nodes

In [32]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Check if the first node is the desired node then search the other nodes
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
first_node_cid = linked_cids[0]
first_node = ipfs.retrieve(first_node_cid)
if first_node.get_content() == target_content:
    print(f"Found node: {node}")
else:
    # run through the rest of the linked cids list to check
    for cid in linked_cids:
        node = ipfs.retrieve(cid)
        if node.get_content() == target_content:
            break
    # Get the linked cids of the node at index 1
    node = ipfs.retrieve(linked_cids[1])
    linked_cids = node.get_linked_cids()
    while True:
        # Flag to determine if we should exit the while loop
        found = False

        # Run through the rest of the linked CIDs list to check
        for cid in linked_cids[1:]:
            node = ipfs.retrieve(cid)
            if node.get_content() == target_content:
                found = True
                break

        if found:
            break  # Exit the while loop if the node was found

        if len(linked_cids) >= 2:
            node = ipfs.retrieve(linked_cids[1])
            linked_cids = node.get_linked_cids()
        else:
            print('Can\'t find node')
            break

# Output the found node
print(f"Found node: {node}")
get_op_counts()

Looking for node with content: Node 13
Can't find node
Found node: {'CID': 'Qm8f80bd8bb8fc158e89c4b9c8ad9559a0fa', 'Content': 'Node 1', 'Linked CID(s)': ['Qmc06f910f97a2c73fb6fbde6e3b1e7ce756'], 'Timestamp': 1737222966.4235377}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 28}


### Linking to Sequential Exponential (Base K, K an integer)

#### Storing nodes

In [107]:
ipfs.reset_data()

# Testing parameters
NODE_NUM = 30
K = 2
URL = "example.com"

# Create and store the first node
content = "Node 0"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
first_node = IPARO(cid=cid, linked_cids=[],
                   content=content, timestamp=timestamp)
ipfs.store(cid, first_node)
ipns.update(URL, cid)

# Create and store the second node
content = "Node 1"
timestamp = time.time()
to_be_hashed = str({
    "content": content,
    "timestamp": timestamp
})
cid = ipfs.hash(to_be_hashed)
linked_cids = [ipns.get_cid(URL)]
second_node = IPARO(cid=cid, linked_cids=linked_cids,
                    content=content, timestamp=timestamp)
ipfs.store(cid, second_node)
ipns.update(URL, cid)

# To automate creating and adding the remaining nodes
for i in range(2, NODE_NUM):
    content = "Node " + str(i)
    print(content)
    timestamp = time.time()
    to_be_hashed = str({
        "content": content,
        "timestamp": timestamp
    })
    cid = ipfs.hash(to_be_hashed)
    latest_node_cid = ipns.get_cid(URL)
    latest_node = ipfs.retrieve(latest_node_cid)
    latest_node_linked_cids = latest_node.get_linked_cids()
    linked_cids = []

    # Link to previous node FIRST
    linked_cids.append(latest_node_linked_cids[0])
    cids = []

    j = 1
    # Temp node = latest node CID inserted (before this one). Call that node K-1.
    if i == 4:
        pass
    temp_node = latest_node
    print(temp_node.get_content())
    done = False
    while True:
        # Theorem: For any m < len(temp_node_linked_cids), the mth-to-last position of the linked CIDs list will link K^m nodes away from the node.
        # Base Case: The previous node always gets assigned the last position (m=0), so the base case holds.
        # Inducive Case: Suppose that the property holds for m=k. Then, we need to prove that it holds for m=k+1.
        # The start node is K^k nodes away from the most recent node. But if we use the kth position of the linked
        # CIDs, and travelled the kth link (K - 1) times, then the CID to be added to the linked CIDs (at the (k+1)th
        # position) is K^k + (K-1) * K^k = K*K^k = K^(k+1), which proves the inductive case.
        # Therefore, by the Principle of Mathematical Induction, I proved that this theorem holds.
        for _ in range(K - 1):
            temp_cid = temp_node.get_cid()
            temp_node_linked_cids = temp_node.get_linked_cids()
            done = j > len(temp_node_linked_cids)
            if done:
                break
            temp_node = ipfs.retrieve(temp_node_linked_cids[-j])
        if done:
            break
        print(temp_node.get_content())
        # Ensure no duplicate links
        if temp_node.get_cid() not in linked_cids:
            cids.append(temp_node.get_cid())
        j += 1
    cids = list(reversed(cids))
    linked_cids.extend(cids)
    linked_cids.append(latest_node_cid)

    print("Length of linked_cids: ", len(linked_cids))
    node = IPARO(cid=cid, linked_cids=linked_cids,
                 content=content, timestamp=timestamp)
    ipfs.store(cid, node)
    ipns.update(URL, cid)

get_op_counts()  # Output operation counts

Node 2
Node 1
Node 0
Length of linked_cids:  2
Node 3
Node 2
Node 1
Length of linked_cids:  3
Node 4
Node 3
Node 2
Node 0
Length of linked_cids:  3
Node 5
Node 4
Node 3
Node 1
Length of linked_cids:  4
Node 6
Node 5
Node 4
Node 2
Length of linked_cids:  4
Node 7
Node 6
Node 5
Node 3
Node 0
Length of linked_cids:  4
Node 8
Node 7
Node 6
Node 4
Node 0
Length of linked_cids:  4
Node 9
Node 8
Node 7
Node 5
Node 1
Length of linked_cids:  5
Node 10
Node 9
Node 8
Node 6
Node 2
Length of linked_cids:  5
Node 11
Node 10
Node 9
Node 7
Node 3
Length of linked_cids:  5
Node 12
Node 11
Node 10
Node 8
Node 4
Length of linked_cids:  5
Node 13
Node 12
Node 11
Node 9
Node 5
Node 0
Length of linked_cids:  5
Node 14
Node 13
Node 12
Node 10
Node 6
Node 0
Length of linked_cids:  5
Node 15
Node 14
Node 13
Node 11
Node 7
Node 0
Length of linked_cids:  5
Node 16
Node 15
Node 14
Node 12
Node 8
Node 0
Length of linked_cids:  5
Node 17
Node 16
Node 15
Node 13
Node 9
Node 1
Length of linked_cids:  6
Node 18
Node 

In [112]:
# Reset the operation counts
ipfs.reset_counts()
ipns.reset_counts()

# Pick a random node to search for
node_num = random.randint(0, NODE_NUM - 1)
target_content = f"Node {node_num}"
print(f"Looking for node with content: {target_content}")

# Check if the first node is the desired node then search the other nodes
latest_node_cid = ipns.get_cid(URL)
node = ipfs.retrieve(latest_node_cid)
linked_cids = node.get_linked_cids()
first_node_cid = linked_cids[0]
first_node = ipfs.retrieve(first_node_cid)
if first_node.get_content() == target_content:
    print(f"Found node: {node}")
else:
    # run through the rest of the linked cids list to check
    for cid in linked_cids:
        node = ipfs.retrieve(cid)
        if node.get_content() == target_content:
            break
    # Get the linked cids of the node at index -1 (where the latest index is)
    node = ipfs.retrieve(linked_cids[-1])
    linked_cids = node.get_linked_cids()
    while True:
        # Flag to determine if we should exit the while loop
        found = False

        # Run through the rest of the linked CIDs list to check
        for cid in linked_cids[1:]:
            node = ipfs.retrieve(cid)
            if node.get_content() == target_content:
                found = True
                break

        if found:
            print(f"Found node: {node}")
            break  # Exit the while loop if the node was found

        if len(linked_cids) >= 2:
            node = ipfs.retrieve(linked_cids[-1])
            linked_cids = node.get_linked_cids()
            # Output the found node
        else:
            print('Can\'t find node')
            break

get_op_counts()

Looking for node with content: Node 22
Found node: {'CID': 'Qm8ad3cc9dd2f9b51f6b034a738f478f11ac', 'Content': 'Node 22', 'Linked CID(s)': ['Qm126aff12842a9bc50dbdbdbf460181654f', 'Qmf3fd0d4d150c51253551f79294ef6eab61', 'Qm375389864575456cbde778569832f00e90', 'Qmab9f7e5e2dc4cb1cf28a9c6c734563e83c', 'Qm831c533fa4968330bb63faabb24b448d32', 'Qme5685c8c8b572c2a5386b4c532ae767234'], 'Timestamp': 1737246260.5532575}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 24}


Looking for node with content: Node 24
Can't find node
Found node: {'CID': 'Qm3e86b1fea41595c77a61345510bb3503b1', 'Content': 'Node 1', 'Linked CID(s)': ['Qme931ce5daa92fe0ad27cc618145ed41c1b'], 'Timestamp': 1737245623.6878963}
Number of operations IPNS performed:
{'get': 1, 'update': 0}
Number of operations IPFS performed:
{'hash': 0, 'store': 0, 'retrieve': 10}


### Other strategies to be tested